In [16]:
# BaseEstimator 커스텀
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    #fit() 메서드 아무 학습 X
    def fit(self , X, y =None): pass
        
    #predict() 는 단순히 sex 피처가 1이면 0 아니면 1 예측
    def predict(self,X):
        pred = np.zeros((X.shape[0],1))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i]==1:
                pred[i]=0
            else :
                pred[i]=1

        return pred
        
from sklearn.preprocessing import LabelEncoder
def transform_features(df):
    # 1. 불필요한 피처 제거
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    
    # 2. 결측치 처리
    df['Age']=df['Age'].fillna(df['Age'].mean())
    df['Cabin']=df['Cabin'].fillna('N')
    df['Embarked']= df['Embarked'].fillna('N')
    
    # 3. 레이블 인코딩 (Sex: 'male'->1, 'female'->0)
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
        
    return df

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터 재로딩 / 가공 / 학습테스트 데이터 분할
titanic_df = pd.read_csv('./train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train , X_test , y_train , y_test = train_test_split(X_titanic_df,y_titanic_df,test_size=0.2,random_state=0)

myclf = MyDummyClassifier()
myclf.fit(X_train,y_train)

mypredictions = myclf.predict(X_test)
print('dummy classifier 정확도 :{0:.4f}'.format(accuracy_score(y_test,mypredictions)))

dummy classifier 정확도 :0.7877


In [33]:
# 정밀도 / 재현율 
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix

def get_clf_eval(y_test,pred=None,pred_proba_=None):
    confusion = confusion_matrix(y_test,pred) # 오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) #재현율 
    f1=f1_score(y_test,ped)
    roc_auc =roc_auc_score(y_test,pred_proba)
    print('오차행렬')
    print(confusion)
    print( '정확도: {0:.4f}, 정 밀도: {1:.4f}, 재현율: {2:.4f},f1:{3:.4f},AUC:{4:.4f}'.format(accuracy,precision, recall,f1,roc_auc)) 

In [27]:
#로지스틱 회귀 기반 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터를 재로딩 , 데이터 가공 , 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('./train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train , X_test , y_train , y_test = train_test_split(X_titanic_df,y_titanic_df,test_size=0.2,random_state=11)

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train,y_train)
pred = lr_clf.predict(X_test)

get_clf_eval(y_test,pred)

오차행렬
[[108  10]
 [ 16  45]]
정확도: 0.8547, 정 밀도: 0.8182, 재현율: 0.7377


In [30]:
# ROC 곡선
from sklearn.metrics import roc_curve

#레이블 값이 1일때의 예측 확률 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]

# 반환값 fpr , tpr , thresholds array 들 
fprs , tprs , thresholds = roc_curve(y_test,pred_proba_class1)

#인덱스 추출
thr_index = np.arange(1,thresholds.shape[0],5)
print('샘플 추출 위한 임계값 배열 index',thr_index)
print('index로 추출한 진짜 임계값',np.round(thresholds[thr_index],2))

# 5step 단위로 추출된 임계값에 따른 rpf tpr값

print('fpr:', np.round(fprs[thr_index], 3))
print('tpr:', np.round(tprs[thr_index], 3))

샘플 추출 위한 임계값 배열 index [ 1  6 11 16 21 26 31 36 41 46 51]
index로 추출한 진짜 임계값 [0.94 0.69 0.62 0.57 0.44 0.36 0.32 0.15 0.13 0.13 0.02]
fpr: [0.    0.008 0.025 0.059 0.11  0.195 0.22  0.534 0.619 0.78  1.   ]
tpr: [0.016 0.525 0.689 0.721 0.787 0.82  0.885 0.902 0.967 0.967 1.   ]


In [32]:
# ROC AUC 성능 수치
from sklearn.metrics import roc_auc_score
pred_proba = lr_clf.predict_proba(X_test)[:,1]
roc_score = roc_auc_score(y_test,pred_proba)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

ROC AUC 값: 0.8980
